# Navigation Patterns

> Protocols and implementations for keyboard navigation within focus zones.

In [ ]:
#| default_exp core.navigation

In [ ]:
#| export
from __future__ import annotations
from dataclasses import dataclass
from typing import Literal, Protocol, runtime_checkable

## Direction Type

Navigation directions supported by the framework.

In [ ]:
#| export
Direction = Literal["up", "down", "left", "right"]

## NavigationPattern Protocol

The base protocol that all navigation patterns must implement.

In [ ]:
#| export
@runtime_checkable
class NavigationPattern(Protocol):
    """Protocol for navigation within a focus zone."""

    @property
    def name(self) -> str: # unique identifier for this pattern
        """Return the pattern name."""
        ...

    def get_next_index(
        self,
        current: int,      # current focused index
        direction: Direction, # navigation direction
        total: int,        # total number of items
        columns: int = 1   # number of columns (for grid navigation)
    ) -> int:              # the new index after navigation
        """Calculate next index given current position and direction."""
        ...

    def get_supported_directions(self) -> tuple[Direction, ...]: # directions this pattern responds to
        """Return which arrow key directions this pattern handles."""
        ...

## LinearVertical

Up/Down navigation through a vertical list of items. This is the most common pattern.

In [ ]:
#| export
@dataclass
class LinearVertical:
    """Up/Down navigation through a vertical list."""
    wrap: bool = False # wrap from last item to first (and vice versa)

    @property
    def name(self) -> str:
        """Return the pattern name."""
        return "linear_vertical"

    def get_supported_directions(self) -> tuple[Direction, ...]: # ("up", "down")
        """Return supported directions."""
        return ("up", "down")

    def get_next_index(
        self,
        current: int,      # current focused index
        direction: Direction, # "up" or "down"
        total: int,        # total number of items
        columns: int = 1   # unused for linear navigation
    ) -> int:              # the new index
        """Calculate next index for vertical navigation."""
        if total == 0:
            return 0
        if direction == "down":
            new = current + 1
            if new >= total:
                return 0 if self.wrap else total - 1
            return new
        elif direction == "up":
            new = current - 1
            if new < 0:
                return total - 1 if self.wrap else 0
            return new
        return current

In [ ]:
# Test LinearVertical
nav = LinearVertical()
assert nav.name == "linear_vertical"
assert nav.get_supported_directions() == ("up", "down")
assert nav.get_next_index(0, "down", 5) == 1
assert nav.get_next_index(4, "down", 5) == 4  # no wrap
assert nav.get_next_index(0, "up", 5) == 0    # no wrap

# Test with wrap
nav_wrap = LinearVertical(wrap=True)
assert nav_wrap.get_next_index(4, "down", 5) == 0  # wraps to start
assert nav_wrap.get_next_index(0, "up", 5) == 4    # wraps to end

## LinearHorizontal

Left/Right navigation through a horizontal list. Useful for word tokens in split mode, tabs, etc.

In [ ]:
#| export
@dataclass
class LinearHorizontal:
    """Left/Right navigation through a horizontal list."""
    wrap: bool = False # wrap from last item to first (and vice versa)

    @property
    def name(self) -> str:
        """Return the pattern name."""
        return "linear_horizontal"

    def get_supported_directions(self) -> tuple[Direction, ...]: # ("left", "right")
        """Return supported directions."""
        return ("left", "right")

    def get_next_index(
        self,
        current: int,      # current focused index
        direction: Direction, # "left" or "right"
        total: int,        # total number of items
        columns: int = 1   # unused for linear navigation
    ) -> int:              # the new index
        """Calculate next index for horizontal navigation."""
        if total == 0:
            return 0
        if direction == "right":
            new = current + 1
            if new >= total:
                return 0 if self.wrap else total - 1
            return new
        elif direction == "left":
            new = current - 1
            if new < 0:
                return total - 1 if self.wrap else 0
            return new
        return current

In [ ]:
# Test LinearHorizontal
nav = LinearHorizontal()
assert nav.name == "linear_horizontal"
assert nav.get_supported_directions() == ("left", "right")
assert nav.get_next_index(0, "right", 5) == 1
assert nav.get_next_index(4, "right", 5) == 4  # no wrap
assert nav.get_next_index(0, "left", 5) == 0   # no wrap

# Test with wrap
nav_wrap = LinearHorizontal(wrap=True)
assert nav_wrap.get_next_index(4, "right", 5) == 0  # wraps
assert nav_wrap.get_next_index(0, "left", 5) == 4   # wraps

## ScrollOnly

A navigation pattern for zones that are scrollable but don't have selectable items (e.g., a preview panel).

In [ ]:
#| export
@dataclass
class ScrollOnly:
    """No item navigation, zone is scrollable content only."""

    @property
    def name(self) -> str:
        """Return the pattern name."""
        return "scroll_only"

    def get_supported_directions(self) -> tuple[Direction, ...]: # empty tuple
        """Return no supported directions."""
        return ()

    def get_next_index(
        self,
        current: int,      # current index (unused)
        direction: Direction, # direction (unused)
        total: int,        # total items (unused)
        columns: int = 1   # columns (unused)
    ) -> int:              # always returns current
        """Return current index unchanged."""
        return current

In [ ]:
# Test ScrollOnly
nav = ScrollOnly()
assert nav.name == "scroll_only"
assert nav.get_supported_directions() == ()
assert nav.get_next_index(2, "down", 5) == 2  # unchanged

## Grid (Placeholder)

2D grid navigation for use in media galleries and similar UIs. Marked as placeholder for future implementation.

In [ ]:
#| export
@dataclass
class Grid:
    """2D grid navigation (placeholder for future implementation)."""
    columns: int = 4           # number of columns in the grid
    wrap_horizontal: bool = True  # wrap at row edges
    wrap_vertical: bool = False   # wrap at grid top/bottom

    @property
    def name(self) -> str:
        """Return the pattern name."""
        return "grid"

    def get_supported_directions(self) -> tuple[Direction, ...]: # all four directions
        """Return all four directions."""
        return ("up", "down", "left", "right")

    def get_next_index(
        self,
        current: int,      # current focused index
        direction: Direction, # navigation direction
        total: int,        # total number of items
        columns: int = 0   # override columns (0 = use self.columns)
    ) -> int:              # the new index
        """Calculate next index for 2D grid navigation."""
        if total == 0:
            return 0
        
        cols = columns if columns > 0 else self.columns
        row, col = divmod(current, cols)
        rows = (total + cols - 1) // cols  # ceiling division

        if direction == "right":
            new_col = col + 1
            if new_col >= cols:
                new_col = 0 if self.wrap_horizontal else cols - 1
            new_idx = row * cols + new_col
            return min(new_idx, total - 1)
        
        elif direction == "left":
            new_col = col - 1
            if new_col < 0:
                new_col = cols - 1 if self.wrap_horizontal else 0
            new_idx = row * cols + new_col
            return min(new_idx, total - 1)
        
        elif direction == "down":
            new_row = row + 1
            if new_row >= rows:
                new_row = 0 if self.wrap_vertical else rows - 1
            new_idx = new_row * cols + col
            return min(new_idx, total - 1)
        
        elif direction == "up":
            new_row = row - 1
            if new_row < 0:
                new_row = rows - 1 if self.wrap_vertical else 0
            new_idx = new_row * cols + col
            return min(new_idx, total - 1)
        
        return current

In [ ]:
# Test Grid navigation
# Grid with 4 columns, 10 items:
# [0, 1, 2, 3]
# [4, 5, 6, 7]
# [8, 9]
grid = Grid(columns=4)
assert grid.name == "grid"
assert grid.get_supported_directions() == ("up", "down", "left", "right")

# Horizontal navigation
assert grid.get_next_index(0, "right", 10) == 1
assert grid.get_next_index(3, "right", 10) == 0  # wraps
assert grid.get_next_index(0, "left", 10) == 3   # wraps

# Vertical navigation
assert grid.get_next_index(1, "down", 10) == 5
assert grid.get_next_index(5, "up", 10) == 1
assert grid.get_next_index(9, "down", 10) == 9   # no vertical wrap by default

## Protocol Verification

In [ ]:
# Verify all implementations satisfy the protocol
assert isinstance(LinearVertical(), NavigationPattern)
assert isinstance(LinearHorizontal(), NavigationPattern)
assert isinstance(ScrollOnly(), NavigationPattern)
assert isinstance(Grid(), NavigationPattern)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()